In [2]:
import yfinance as yf
import pandas as pd
import os
from datetime import datetime

def calcular_dividendos_5_anos(ticker_obj: yf.Ticker) -> float:
    """
    Soma os dividendos pagos nos últimos 5 anos, corrigindo problemas de fuso horário.
    """
    try:
        dividends = ticker_obj.dividends
        if dividends.empty:
            return 0.0
        # Remove o fuso horário do índice
        dividends.index = dividends.index.tz_convert(None)
        limite = datetime.now()
        inicio = limite - pd.Timedelta(days=5*365)
        dividendos_5_anos = dividends[dividends.index >= inicio]
        return round(dividendos_5_anos.sum(), 4)
    except Exception as e:
        print(f" (Aviso: Falha ao calcular dividendos para {ticker_obj.ticker} - {e}) ", end="")
        return 0.0


def fetch_stock_data(ticker: str, info_original: pd.Series) -> dict:
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period="5y")

    market_cap = info.get('marketCap')
    current_price = info.get("currentPrice", 0.0)
    if not market_cap or market_cap == 0:
        shares_outstanding = info.get('sharesOutstanding', 0)
        if shares_outstanding > 0 and current_price > 0:
            market_cap = shares_outstanding * current_price
    if not market_cap or market_cap == 0:
        market_cap = info_original.get('Market Cap', 0)

    dividendos_5_anos = calcular_dividendos_5_anos(stock)

    dados = {
        "Empresa": info_original.get('Empresa', 'N/A'),
        "Ticker": ticker,
        "Setor": info_original.get('Setor (brapi)', 'N/A'),
        "Tipo": info_original.get('Tipo', 'N/A'),
        "Preço Atual": current_price,
        "Market Cap": market_cap,
        "Dividendos 5 Anos (R$)": dividendos_5_anos
    }
    return dados

def main():
    arquivo_input = r"E:\Github\Unicamp\finance-manager\data\scanner_acoes_e_fundos_filtrado.csv"
    output_path = r"E:\Github\Unicamp\finance-manager\data\relatorio_dividendos_5_anos.csv"

    try:
        df_input = pd.read_csv(arquivo_input)
        tickers_para_analisar = df_input['Ticker'].dropna().unique().tolist()
    except FileNotFoundError:
        print(f"❌ Arquivo '{arquivo_input}' não encontrado.")
        return

    dados_detalhados = {}
    print("\n🔍 Iniciando análise de dividendos dos últimos 5 anos...")
    for ticker in tickers_para_analisar:
        print(f"Processando: {ticker}...", end="")
        try:
            info_original = df_input[df_input['Ticker'] == ticker].iloc[0]
            dados_detalhados[ticker] = fetch_stock_data(ticker, info_original)
            print(" ✅")
        except Exception as e:
            print(f" ❌ (Erro: {e})")

    if not dados_detalhados:
        print("\n⚠️ Nenhum dado coletado com sucesso.")
        return

    df_resultado = pd.DataFrame.from_dict(dados_detalhados, orient="index")
    df_resultado.fillna(0, inplace=True)

    try:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        df_resultado.to_csv(output_path, index=False)
        print(f"\n📁 Relatório salvo com sucesso em: {output_path}")
    except Exception as e:
        print(f"❌ Erro ao salvar o arquivo: {e}")

if __name__ == "__main__":
    main()



🔍 Iniciando análise de dividendos dos últimos 5 anos...
Processando: VIVT3.SA... ✅
Processando: TIMS3.SA... ✅
Processando: FIQE3.SA... ✅
Processando: BRST3.SA... ✅
Processando: DESK3.SA... ✅
Processando: TELB4.SA... ✅
Processando: TELB3.SA... ✅
Processando: OIBR3.SA... ✅
Processando: OIBR4.SA... ✅
Processando: ITUB3.SA... ✅
Processando: ITUB4.SA... ✅
Processando: BPAC3.SA... ✅
Processando: BPAC5.SA... ✅
Processando: BBDC3.SA... ✅
Processando: BBDC4.SA... ✅
Processando: BPAC11.SA... ✅
Processando: SANB11.SA... ✅
Processando: BBAS3.SA... ✅
Processando: ITSA4.SA... ✅
Processando: ITSA3.SA... ✅
Processando: SANB3.SA... ✅
Processando: SANB4.SA... ✅
Processando: B3SA3.SA... ✅
Processando: BBSE3.SA... ✅
Processando: CXSE3.SA... ✅
Processando: RENT3.SA... ✅
Processando: PSSA3.SA... ✅
Processando: MULT3.SA... ✅
Processando: ALOS3.SA... ✅
Processando: BPAN4.SA... ✅
Processando: CYRE3.SA... ✅
Processando: BNBR3.SA... ✅
Processando: CURY3.SA... ✅
Processando: DIRR3.SA... ✅
Processando: IGTI3.SA..